In [119]:
import os
import pandas as pd
from robustness.tools.breeds_helpers import setup_breeds
from robustness.tools.breeds_helpers import ClassHierarchy
import numpy as np
import json


In [142]:
data_dir = '/tmp/imagenet'
info_dir = './imagenet_class_hierarchy/modified'

In [149]:
# Information (names, IDs) for intermediate nodes in hierarchy.
with open(os.path.join(info_dir, "node_names.txt")) as f:
    mapping = [l.strip().split('\t') for l in f.readlines()]

In [157]:
if not (os.path.exists(info_dir) and len(os.listdir(info_dir))):
    print("Downloading class hierarchy information into `info_dir`")
    setup_breeds(info_dir)

hier = ClassHierarchy(info_dir)

In [144]:
with open(os.path.join(info_dir, "dataset_class_info.json")) as f:
            class_info = json.load(f)

nodes_1k = [row[1] for row in class_info]


In [172]:
prompts = {}
count = 0
for i in range(1, 9):
    nodes = hier.get_nodes_at_level(i)
    for node in nodes:
        node_name = hier.HIER_NODE_NAME[node].partition(",")[0]
        node_details = [node, f"a photo of a {node_name}.", node_name]
        leaves = hier.leaves_reachable(node)
        if len(leaves) == 1 and next(iter(leaves))==node:
            if node not in nodes_1k:
                continue
            is_leaf = 1
        else:
            is_leaf = 0
        node_details.append(is_leaf)
        cls_idx = hier.LEAF_ID_TO_NUM[node] if is_leaf==1 else 99999
        node_details.append(cls_idx)
        prompts[count] = node_details
        count += 1

prompts_df = pd.DataFrame.from_dict(prompts, orient='index', columns=['node', 'prompt', 'class_name', 'is_leaf', 'class_idx'])
prompts_df.insert(0, 'index', range(0, 0+len(prompts_df)))
# prompts_df.to_csv("imagenet_hierarchy_complete.csv", index=False, header=True)


In [154]:
prompts_df.to_csv("imagenet_hierarchy_complete.csv", index=False, header=True)

In [180]:
nodes = hier.get_nodes_at_level(3)

for node in nodes:
        node_name = hier.HIER_NODE_NAME[node].partition(",")[0]
        leaves = hier.leaves_reachable(node)
        print(f"{node_name}, leaves: {leaves}")
        if len(leaves) == 1 and next(iter(leaves))==node:
                print("leaf")
        else:
                print(f"{node_name} not leaf")


mollusk, leaves: {'n01944390', 'n01955084', 'n01968897', 'n01950731', 'n01945685', 'n01943899'}
mollusk not leaf
bird, leaves: {'n01829413', 'n01608432', 'n01622779', 'n01580077', 'n02033041', 'n01616318', 'n01534433', 'n01806143', 'n01806567', 'n01796340', 'n01798484', 'n01592084', 'n02056570', 'n01855032', 'n02018207', 'n01833805', 'n01795545', 'n01514668', 'n01843383', 'n01532829', 'n01843065', 'n02002724', 'n01531178', 'n01558993', 'n01818515', 'n01560419', 'n02009229', 'n01824575', 'n01807496', 'n01601694', 'n02013706', 'n02025239', 'n02009912', 'n01820546', 'n01614925', 'n02018795', 'n01819313', 'n01828970', 'n01582220', 'n02058221', 'n01514859', 'n02006656', 'n02002556', 'n01817953', 'n02037110', 'n02011460', 'n01797886', 'n01518878', 'n02012849', 'n01537544', 'n01855672', 'n02028035', 'n02017213', 'n01530575', 'n01860187', 'n02027492', 'n01847000', 'n02007558', 'n02051845'}
bird not leaf
wheeled vehicle, leaves: {'n03444034', 'n03796401', 'n04465501', 'n03895866', 'n03930630', 

In [177]:
hier.leaves_reachable('n07800740')

TypeError: 'set' object is not subscriptable

In [161]:
# hier.HIER_NODE_NAME["n07800740"]
hier.print_node_info(["n07800740"])

n07800740: fodder (1)


In [ ]:
leaf_nodes = []
visited = []
indices_to_visit = []

while len(leaf_nodes) != 1000:
     # do blah blah with indices to visit
    